In [1]:
# https://github.com/DataTalksClub/llm-zoomcamp/blob/main/03-vector-search/eval/evaluation-metrics.md

In [2]:
2+2

4

In [3]:
import pandas as pd
from tqdm.auto import tqdm
import json
import sys
import os
from dotenv import load_dotenv
# Load environment variables from .env
load_dotenv("/home/jovyan/.envrc")

True

In [4]:
from elasticsearch import Elasticsearch

# Create an instance of the Elasticsearch client
es = Elasticsearch(['http://elasticsearch:9200'])

# Check if the connection is established
if es.ping():
    print("Elasticsearch is connected!")
else:
    print("Elasticsearch is not reachable.")

Elasticsearch is connected!


In [5]:
from sentence_transformers import SentenceTransformer

In [6]:
from elasticsearch import Elasticsearch

In [7]:
# # Directory containing the data files
# data_dir = os.path.abspath('../reviews-assistant/data/reviews')

# # Initialize an empty list to hold all reviews
# reviews = []

# # List objects in the directory
# objects_in_directory = os.listdir(data_dir)

# # Iterate over the files in the directory
# for obj in objects_in_directory:
#     if obj.endswith('.json'):  # Check if the file is a JSON file
#         file_path = os.path.join(data_dir, obj)
#         with open(file_path, 'r', encoding='utf-8') as jsonfile:
#             # Load the reviews from the JSON file
#             file_reviews = json.load(jsonfile)
#             reviews.extend(file_reviews)  # Append reviews to the main list
# # Print the first i reviews
# i = 2  # Change this to print more reviews if needed
# for review in reviews[:i]:
#     print(f"Author ID: {review['author.steamid']}")
#     print(f"Review: {review.get('review', 'No text')}")
#     print(f"Timestamp Created: {review['timestamp_created']}")
#     print("-" * 79)

In [8]:
# len(reviews)

In [9]:
# reviews[1:2]

In [10]:
model_name = 'multi-qa-MiniLM-L6-cos-v1'
model = SentenceTransformer(model_name)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/11.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/opt/conda/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
# for doc in tqdm(reviews):
#     title = doc["title"]
#     review = doc['review']
#     tr = title + ' ' + review

#     doc['title_vector'] = model.encode(title)
#     doc['review_vector'] = model.encode(review)
#     doc['title_review_vector'] = model.encode(tr)

In [12]:
# doc.keys()

In [13]:
from elasticsearch import Elasticsearch, ConnectionError

try:
    es = Elasticsearch(['http://elasticsearch:9200'])  # Use the service name
    if es.ping():
        print("Connected to Elasticsearch")
    else:
        print("Could not connect to Elasticsearch")
except ConnectionError as e:
    print(f"Connection error: {e}")


Connected to Elasticsearch


In [14]:
from elasticsearch import Elasticsearch, NotFoundError, ConnectionError
from tqdm import tqdm

class ReviewIndexer:
    def __init__(self, es_host='http://elasticsearch:9200', index_name='steam-reviews', model=None):
        self.es = Elasticsearch([es_host])
        self.index_name = index_name
        self.model = model  # Expecting a model to encode text
        
        # Check the connection upon initialization
        self.check_connection()
        
        self.index_settings = {
            "settings": {
                "number_of_shards": 1,
                "number_of_replicas": 0
            },
            "mappings": {
                "properties": {
                    "appid": {"type": "keyword"},
                    "timestamp_query": {"type": "integer"},
                    "title": {"type": "keyword"},
                    "recommendationid": {"type": "keyword"},  # Changed to keyword for consistency
                    "author.steamid": {"type": "keyword"},
                    "author.playtimeforever": {"type": "integer"},
                    "author.playtime_last_two_weeks": {"type": "integer"},
                    "author.playtime_at_review": {"type": "integer"},
                    "author.last_played": {"type": "integer"},
                    "language": {"type": "keyword"},
                    "review": {"type": "text"},
                    "voted_up": {"type": "boolean"},  # Changed to boolean for consistency
                    "votes_up": {"type": "integer"},
                    "timestamp_created": {"type": "integer"},
                    "timestamp_updated": {"type": "integer"},
                    "title_vector": {
                        "type": "dense_vector",
                        "dims": 384,
                        "index": True,
                        "similarity": "cosine"
                    },
                    "review_vector": {
                        "type": "dense_vector",
                        "dims": 384,
                        "index": True,
                        "similarity": "cosine"
                    },
                    "title_review_vector": {
                        "type": "dense_vector",
                        "dims": 384,
                        "index": True,
                        "similarity": "cosine"
                    },
                }
            }
        }

    def check_connection(self):
        try:
            if not self.es.ping():
                raise ConnectionError("Elasticsearch cluster is down!")
            print("Successfully connected to Elasticsearch.")
        except ConnectionError as e:
            print(f"Connection error: {e}")
            raise

    def create_index(self):
        try:
            self.es.indices.delete(index=self.index_name, ignore_unavailable=True)
            print(f"Deleted index: {self.index_name} (if it existed)")
        except NotFoundError:
            print(f"Index {self.index_name} does not exist, nothing to delete.")
        except ConnectionError as e:
            print(f"Error deleting index: {e}")

        try:
            self.es.indices.create(index=self.index_name, body=self.index_settings)
            print(f"Created index: {self.index_name}")
        except ConnectionError as e:
            print(f"Error creating index: {e}")

    def prepare_and_index_documents(self, reviews):
        """
        Prepare documents by encoding and indexing them into Elasticsearch.
        
        :param reviews: List of review documents to be indexed.
        """
        for doc in tqdm(reviews):
            # Handle None values for numerical fields
            doc['author.playtimeforever'] = doc.get('author.playtimeforever', 0) or 0
            doc['author.playtime_last_two_weeks'] = doc.get('author.playtime_last_two_weeks', 0) or 0
            doc['author.playtime_at_review'] = doc.get('author.playtime_at_review', 0) or 0
            doc['author.last_played'] = doc.get('author.last_played', 0) or 0
            doc['votes_up'] = doc.get('votes_up', 0) or 0
            doc['timestamp_created'] = doc.get('timestamp_created', 0) or 0
            doc['timestamp_updated'] = doc.get('timestamp_updated', 0) or 0

            # Encode title and review if a model is provided
            if self.model:
                title = doc["title"]
                review = doc['review']
                tr = title + ' ' + review

                # Encode the text and convert to lists
                doc['title_vector'] = self.model.encode(title).tolist()
                doc['review_vector'] = self.model.encode(review).tolist()
                doc['title_review_vector'] = self.model.encode(tr).tolist()
            else:
                # Placeholder if no model is provided
                doc['title_vector'] = []
                doc['review_vector'] = []
                doc['title_review_vector'] = []

            # Index the document
            try:
                self.es.index(index=self.index_name, document=doc)
            except Exception as e:
                print(f"Failed to index document {doc}: {e}")

    def fetch_first_documents(self, size=10):
        body = {
            "query": {
                "match_all": {}
            },
            "size": size
        }

        try:
            response = self.es.search(index=self.index_name, body=body)
            return response['hits']['hits']
        except NotFoundError:
            print(f"Index {self.index_name} not found.")
        except ConnectionError as e:
            print(f"Error searching index: {e}")
        except Exception as e:
            print(f"An error occurred: {e}")
        return None

    def query_reviews(self, title, query, k=5, num_candidates=10000):
        """
        Query reviews using both KNN and keyword search.
    
        :param title: The title of the game to filter results.
        :param query: The natural language query for searching.
        :param k: The number of nearest neighbors to retrieve.
        :param num_candidates: Number of candidates for KNN search.
        :return: The combined search results from KNN and keyword queries.
        """
        # Encode the query
        v_q = self.model.encode(query).tolist()  # Ensure it is a list
    
        # KNN Query
        knn_query = {
            "field": "review_vector",
            "query_vector": v_q,
            "k": k,
            "num_candidates": num_candidates,
            "boost": 0.5,
            "filter": {
                "term": {
                    "title": title  # Using title for filtering
                }
            }
        }
    
        # Keyword Query
        keyword_query = {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["review^3", "title"],
                        "type": "best_fields",
                        "boost": 0.5,
                    }
                },
                "filter": {
                    "term": {
                        "title": title  # Using title for filtering
                    }
                }
            }
        }
    
        # Execute the keyword query
        keyword_results = []
        try:
            keyword_response = self.es.search(index=self.index_name, body={"query": keyword_query}, size=k)
            keyword_results = keyword_response["hits"]["hits"]
        except Exception as e:
            print(f"An error occurred during keyword search: {e}")
    
        # Execute the KNN query
        knn_results = []
        try:
            knn_response = self.es.search(index=self.index_name, body={"knn": knn_query}, size=k)
            knn_results = knn_response["hits"]["hits"]
        except Exception as e:
            print(f"An error occurred during KNN search: {e}")
    
        # Combine results
        combined_results = keyword_results + knn_results  # You may want to deduplicate or sort these results
    
        return combined_results

    def query_reviews_hybrid(self, title, query, k=5, num_candidates=10000, vector_field="review_vector"):
        """
        Query reviews using both KNN and keyword search with an option to select the vector field.
    
        :param title: The title of the game to filter results.
        :param query: The natural language query for searching.
        :param k: The number of nearest neighbors to retrieve.
        :param num_candidates: Number of candidates for KNN search.
        :param vector_field: The field to use for KNN vector search. Can be 'review_vector', 'title_vector', or 'title_review_vector'.
        :return: The combined search results from KNN and keyword queries.
        """
        
        # Validate the vector field
        if vector_field not in ["review_vector", "title_vector", "title_review_vector"]:
            raise ValueError(f"Invalid vector field: {vector_field}. Must be 'review_vector', 'title_vector', or 'title_review_vector'.")
        
        # Encode the query
        v_q = self.model.encode(query).tolist()  # Ensure it is a list
    
        # KNN Query
        knn_query = {
            "field": vector_field,  # Use the selected vector field
            "query_vector": v_q,
            "k": k,
            "num_candidates": num_candidates,
            "boost": 0.5,
            "filter": {
                "term": {
                    "title": title  # Using title for filtering
                }
            }
        }
    
        # Keyword Query
        keyword_query = {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["review^3", "title"],
                        "type": "best_fields",
                        "boost": 0.5,
                    }
                },
                "filter": {
                    "term": {
                        "title": title  # Using title for filtering
                    }
                }
            }
        }
    
        # Execute the keyword query
        keyword_results = []
        try:
            keyword_response = self.es.search(index=self.index_name, body={"query": keyword_query}, size=k)
            keyword_results = keyword_response["hits"]["hits"]
        except Exception as e:
            print(f"An error occurred during keyword search: {e}")
    
        # Execute the KNN query
        knn_results = []
        try:
            knn_response = self.es.search(index=self.index_name, body={"knn": knn_query}, size=k)
            knn_results = knn_response["hits"]["hits"]
        except Exception as e:
            print(f"An error occurred during KNN search: {e}")
    
        # Combine results
        combined_results = keyword_results + knn_results  # You may want to deduplicate or sort these results
    
        return combined_results


    def get_distinct_titles(self):
        """
        Get distinct titles from the Elasticsearch index.
    
        :return: A list of distinct titles.
        """
        try:
            # Define the aggregation query
            aggs = {
                "distinct_titles": {
                    "terms": {
                        "field": "title.keyword",  # Use ".keyword" for exact matches on keyword fields
                        "size": 1000  # Adjust this size as needed
                    }
                }
            }
    
            # Execute the aggregation query
            response = self.es.search(index=self.index_name, body={"size": 0, "aggs": aggs})
            distinct_titles = response['aggregations']['distinct_titles']['buckets']
    
            # Extract titles from the buckets
            titles_list = [bucket['key'] for bucket in distinct_titles]
            return titles_list
    
        except Exception as e:
            print(f"An error occurred while fetching distinct titles: {e}")
            return []


# Usage example
# Assuming you have a list of reviews and a model for encoding
indexer = ReviewIndexer(model=model)  # Pass your model here
indexer.create_index()  # Ensure the index is created first

# Sample input for indexing
# reviews = [
#     {
#         'appid': '721180',
#         'timestamp_query': 1727608376,
#         'title': 'Dustborn',
#         'recommendationid': '173934401',
#         'author.steamid': '76561198028269344',
#         'author.playtimeforever': None,
#         'author.playtime_last_two_weeks': 0,
#         'author.playtime_at_review': 90,
#         'author.last_played': 1725140143,
#         'language': 'english',
#         'review': 'I went into this with an open mind...',
#         'voted_up': False,
#         'votes_up': 401,
#         'timestamp_created': 1725142580,
#         'timestamp_updated': 1725142580,
#         'title_vector': [],  # Initial placeholders
#         'review_vector': [],
#         'title_review_vector': []
#     }
# ]

# # Prepare and index documents
# indexer.prepare_and_index_documents(reviews)

# # Fetch and print the first documents after indexing
# first_documents = indexer.fetch_first_documents(size=10)
# if first_documents:
#     for doc in first_documents:
#         print(doc['_source'])  # Print the document details


Successfully connected to Elasticsearch.
Deleted index: steam-reviews (if it existed)
Created index: steam-reviews


In [15]:
# Directory containing the data files
data_dir = os.path.abspath('../reviews-assistant/data/reviews')

# Initialize an empty list to hold all reviews
reviews = []

# List objects in the directory
objects_in_directory = os.listdir(data_dir)

# Iterate over the files in the directory
for obj in objects_in_directory:
    if obj.endswith('.json'):  # Check if the file is a JSON file
        file_path = os.path.join(data_dir, obj)
        with open(file_path, 'r', encoding='utf-8') as jsonfile:
            # Load the reviews from the JSON file
            file_reviews = json.load(jsonfile)
            reviews.extend(file_reviews)  # Append reviews to the main list
# Print the first i reviews
i = 2  # Change this to print more reviews if needed
for review in reviews[:i]:
    print(f"Author ID: {review['author.steamid']}")
    print(f"Review: {review.get('review', 'No text')}")
    print(f"Timestamp Created: {review['timestamp_created']}")
    print("-" * 79)

Author ID: 76561198420943538
Review: ---{ Graphics }---
✅ You forget what reality is
☐ Beautiful
☐ Good
☐ Decent
☐ Bad
☐ You will get eye cancer
☐ Get a pepper spray for your eye instead

---{ Gameplay }---
☐ Won’t ever touch any other game anymore
✅ Very good
☐ Good
☐ It's just gameplay
☐ Mehh
☐ Watch paint dry instead
☐ Tic Tac toe is better

---{ Audio }---
☐ Eargasm
✅ Very good
☐ Good
☐ Not too bad
☐ Bad
☐ I'm now deaf

---{ Audience }---
☐ Kids
✅Teens
✅ Adults
☐ Grandma

---{ PC Requirements }---
☐ Check if you can run paint
☐ Potato
☐ Decent
✅ Fast
☐ Rich boi
☐ Ask NASA if they have a spare computer
☐ Search the galaxy for dark matter fuel to run

---{ Difficulty }---
☐ Just press 'W'
☐ Easy
✅ Easy to learn / Hard to master
☐ Significant brain usage
☐ Difficult
☐ Dark Souls

---{ Grind }---
☐ Nothing to grind
☐ Only if u care about leaderboards/ranks
✅ Isn't necessary to progress
☐ Average grind level
☐ Too much grind
☐ You'll need a second life for grinding

---{ Story }---
☐ No

In [16]:
len(reviews)

1782

In [17]:
# Prepare and index documents
indexer.prepare_and_index_documents(reviews)

100%|██████████| 1782/1782 [02:35<00:00, 11.46it/s]


In [18]:
# print(doc['_source'].keys())

In [45]:
# Fetch and print the first documents after indexing
first_documents = indexer.fetch_first_documents(size=10)
if first_documents:
    for doc in first_documents:
        print(doc['_source'])  # Print the document details

{'appid': '812140', 'timestamp_query': 1727643597, 'title': "Assassin's Creed Odyssey", 'recommendationid': '174328852', 'author.steamid': '76561198114287599', 'author.playtimeforever': 0, 'author.playtime_last_two_weeks': 719, 'author.playtime_at_review': 7374, 'author.last_played': 1727503878, 'language': 'english', 'review': "I played this a couple years ago. Wanted to play it again but the third party launch Ubisoft Connect isn't working at all. No sign in screen. Nothing. Run as administrator. Nothing. I have also had issues with Origins which I never even got to play on PC because it doesn't work. Everything about Ubisoft is so janky.", 'voted_up': False, 'votes_up': 3, 'timestamp_created': 1725596681, 'timestamp_updated': 1725596681, 'title_vector': [0.01829216629266739, -0.00221725320443511, -0.10289765894412994, -0.07697086781263351, -0.04360188916325569, 0.03980034217238426, 0.006978109013289213, -0.0556001178920269, 0.0761142447590828, 0.01950688473880291, -0.053603217005729

In [20]:
# Create an instance of ReviewIndexer
indexer = ReviewIndexer()

# Get distinct titles
distinct_titles = indexer.get_distinct_titles()

# Print distinct titles
for title in distinct_titles:
    print(title)

Successfully connected to Elasticsearch.


In [21]:
# Usage example
indexer = ReviewIndexer(model=model)  # Ensure your model is passed here
# indexer.create_index()  # Ensure the index is created first

# Sample input for querying
title = "God of War: Ragnarok"  # Example game title
query = 'Is God of War: Ragnarok a game for kids?'

# Perform the query
results = indexer.query_reviews(title, query)

# Convert the results to a pandas DataFrame
if results:  # Check if results are not empty
    # Extract the source data from the Elasticsearch response
    data = [doc['_source'] for doc in results]
    
    # Create a DataFrame
    df_results = pd.DataFrame(data)
    
    # Set pandas options to display full content of the 'review' column
    # pd.set_option('display.max_colwidth', None)  # or use a larger number to set a specific width
    # pd.set_option('display.max_rows', None)  # Show all rows if needed
    pd.reset_option('all')

    # Display the DataFrame
    # print(df_results.head())  # or use df_results to view the entire DataFrame
else:
    print("No results found.")

Successfully connected to Elasticsearch.


/tmp/ipykernel_652/1461783233.py:187: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  keyword_response = self.es.search(index=self.index_name, body={"query": keyword_query}, size=k)
/tmp/ipykernel_652/1461783233.py:195: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  knn_response = self.es.search(index=self.index_name, body={"knn": knn_query}, size=k)
/tmp/ipykernel_652/2173238283.py:23: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  pd.reset_option('all')


In [22]:
df_results

,appid,timestamp_query,title,recommendationid,author.steamid,author.playtimeforever,author.playtime_last_two_weeks,author.playtime_at_review,author.last_played,language,review,voted_up,votes_up,timestamp_created,timestamp_updated,title_vector,review_vector,title_review_vector
0,2322010,1727643597,God of War: Ragnarok,175347860,76561198354466742,0,827,250,1727081833,english,"to God of War: Ragnarok Dev. Team,\n\nI’m writ...",True,989,1726803482,1726816251,"[-0.030018333345651627, 0.03668512776494026, -...","[0.07209023833274841, -0.03378300368785858, -0...","[0.07215426117181778, -0.027821576222777367, -..."
1,2322010,1727643597,God of War: Ragnarok,175313650,76561198053124774,0,80,72,1727116275,english,God of War Ragnarok could have been an unforge...,False,1080,1726761245,1726884791,"[-0.030018333345651627, 0.03668512776494026, -...","[-0.027094047516584396, -0.07563106715679169, ...","[-0.029218800365924835, -0.06694116443395615, ..."
2,2322010,1727643597,God of War: Ragnarok,175731340,76561199237205683,0,3321,2062,1727606415,english,"If you played God of War (2018) and liked it, ...",True,11,1727249712,1727249712,"[-0.030018333345651627, 0.03668512776494026, -...","[0.011011939495801926, -0.039762064814567566, ...","[0.018289970234036446, -0.027919186279177666, ..."
3,2322010,1727643597,God of War: Ragnarok,175635671,76561199170479681,0,1555,303,1727565301,english,Why is this 100% single player experience forc...,False,261,1727120061,1727217923,"[-0.030018333345651627, 0.03668512776494026, -...","[0.009475771337747574, -0.08857275545597076, 0...","[0.02115711197257042, -0.08730130642652512, 0...."
4,2322010,1727643597,God of War: Ragnarok,175320197,76561198342881763,0,1011,60,1727471180,english,"The days of searching for ""Spider-Man PC, Unch...",True,401,1726767566,1726767566,"[-0.030018333345651627, 0.03668512776494026, -...","[-0.05745435878634453, -0.029408765956759453, ...","[-0.049518052488565445, -0.01963861472904682, ..."
5,2322010,1727643597,God of War: Ragnarok,175731340,76561199237205683,0,3321,2062,1727606415,english,"If you played God of War (2018) and liked it, ...",True,11,1727249712,1727249712,"[-0.030018333345651627, 0.03668512776494026, -...","[0.011011939495801926, -0.039762064814567566, ...","[0.018289970234036446, -0.027919186279177666, ..."
6,2322010,1727643597,God of War: Ragnarok,175310996,76561198124407909,0,4221,6,1727449949,english,"[h1]A Great Sequel, Held Back by Woke[/h1]\n\n...",True,384,1726758795,1726764427,"[-0.030018333345651627, 0.03668512776494026, -...","[-0.035042811185121536, -0.041156742721796036,...","[-0.02929731272161007, -0.034129079431295395, ..."
7,2322010,1727643597,God of War: Ragnarok,175699724,76561198089929485,0,2681,2605,1727214509,english,after hearing so many negative things about ra...,True,19,1727204312,1727204312,"[-0.030018333345651627, 0.03668512776494026, -...","[-0.021168339997529984, -0.013783842325210571,...","[-0.00749595882371068, 0.014234294183552265, 0..."
8,2322010,1727643597,God of War: Ragnarok,175324218,76561198191868681,0,695,68,1727606965,english,"Loved the first God of War, played it multiple...",True,3,1726771543,1726772222,"[-0.030018333345651627, 0.03668512776494026, -...","[0.05148572474718094, -0.01094889733940363, -5...","[0.02400249056518078, -0.0324452705681324, -0...."
9,2322010,1727643597,God of War: Ragnarok,175736674,76561197984102076,0,568,568,1727002220,english,a children's game for little babies. every puz...,False,13,1727259027,1727259092,"[-0.030018333345651627, 0.03668512776494026, -...","[-0.05048228055238724, -0.016842171549797058, ...","[-0.03342920169234276, -0.018790561705827713, ..."


In [23]:
# Usage example
indexer = ReviewIndexer(model=model)  # Ensure your model is passed here
# indexer.create_index()  # Ensure the index is created first

# Sample input for querying
title = "God of War: Ragnarok"  # Example game title
query = 'Is God of War: Ragnarok a game for kids?'

# Perform the hybrid query using title_review_vector
results = indexer.query_reviews_hybrid(title, query, vector_field="title_review_vector")

# Convert the results to a pandas DataFrame
if results:  # Check if results are not empty
    # Extract the source data from the Elasticsearch response
    data = [doc['_source'] for doc in results]
    
    # Create a DataFrame
    df_results = pd.DataFrame(data)
    
    # Set pandas options to display full content of the 'review' column
    # pd.set_option('display.max_colwidth', None)  # or use a larger number to set a specific width
    # pd.set_option('display.max_rows', None)  # Show all rows if needed
    pd.reset_option('all')

    # Display the DataFrame
    # print(df_results.head())  # or use df_results to view the entire DataFrame
else:
    print("No results found.")

Successfully connected to Elasticsearch.


/tmp/ipykernel_652/1461783233.py:260: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  keyword_response = self.es.search(index=self.index_name, body={"query": keyword_query}, size=k)
/tmp/ipykernel_652/1461783233.py:268: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  knn_response = self.es.search(index=self.index_name, body={"knn": knn_query}, size=k)
/tmp/ipykernel_652/2462003919.py:23: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  pd.reset_option('all')


In [24]:
df_results

,appid,timestamp_query,title,recommendationid,author.steamid,author.playtimeforever,author.playtime_last_two_weeks,author.playtime_at_review,author.last_played,language,review,voted_up,votes_up,timestamp_created,timestamp_updated,title_vector,review_vector,title_review_vector
0,2322010,1727643597,God of War: Ragnarok,175347860,76561198354466742,0,827,250,1727081833,english,"to God of War: Ragnarok Dev. Team,\n\nI’m writ...",True,989,1726803482,1726816251,"[-0.030018333345651627, 0.03668512776494026, -...","[0.07209023833274841, -0.03378300368785858, -0...","[0.07215426117181778, -0.027821576222777367, -..."
1,2322010,1727643597,God of War: Ragnarok,175313650,76561198053124774,0,80,72,1727116275,english,God of War Ragnarok could have been an unforge...,False,1080,1726761245,1726884791,"[-0.030018333345651627, 0.03668512776494026, -...","[-0.027094047516584396, -0.07563106715679169, ...","[-0.029218800365924835, -0.06694116443395615, ..."
2,2322010,1727643597,God of War: Ragnarok,175731340,76561199237205683,0,3321,2062,1727606415,english,"If you played God of War (2018) and liked it, ...",True,11,1727249712,1727249712,"[-0.030018333345651627, 0.03668512776494026, -...","[0.011011939495801926, -0.039762064814567566, ...","[0.018289970234036446, -0.027919186279177666, ..."
3,2322010,1727643597,God of War: Ragnarok,175635671,76561199170479681,0,1555,303,1727565301,english,Why is this 100% single player experience forc...,False,261,1727120061,1727217923,"[-0.030018333345651627, 0.03668512776494026, -...","[0.009475771337747574, -0.08857275545597076, 0...","[0.02115711197257042, -0.08730130642652512, 0...."
4,2322010,1727643597,God of War: Ragnarok,175320197,76561198342881763,0,1011,60,1727471180,english,"The days of searching for ""Spider-Man PC, Unch...",True,401,1726767566,1726767566,"[-0.030018333345651627, 0.03668512776494026, -...","[-0.05745435878634453, -0.029408765956759453, ...","[-0.049518052488565445, -0.01963861472904682, ..."
5,2322010,1727643597,God of War: Ragnarok,175736674,76561197984102076,0,568,568,1727002220,english,a children's game for little babies. every puz...,False,13,1727259027,1727259092,"[-0.030018333345651627, 0.03668512776494026, -...","[-0.05048228055238724, -0.016842171549797058, ...","[-0.03342920169234276, -0.018790561705827713, ..."
6,2322010,1727643597,God of War: Ragnarok,175723966,76561198032937507,0,958,667,1727565176,english,absolutely love this game! 10x better than the...,True,11,1727236740,1727236740,"[-0.030018333345651627, 0.03668512776494026, -...","[-0.05623815953731537, 0.037968963384628296, 0...","[-0.04380227252840996, 0.007600828073918819, -..."
7,2322010,1727643597,God of War: Ragnarok,175324218,76561198191868681,0,695,68,1727606965,english,"Loved the first God of War, played it multiple...",True,3,1726771543,1726772222,"[-0.030018333345651627, 0.03668512776494026, -...","[0.05148572474718094, -0.01094889733940363, -5...","[0.02400249056518078, -0.0324452705681324, -0...."
8,2322010,1727643597,God of War: Ragnarok,175692493,76561198329851243,0,2684,2461,1727287811,english,"This is more then just a game, it's an experie...",True,12,1727196752,1727196752,"[-0.030018333345651627, 0.03668512776494026, -...","[-0.0069306050427258015, -0.038518667221069336...","[-0.01866106316447258, -0.05488935485482216, -..."
9,2322010,1727643597,God of War: Ragnarok,175697832,76561198263367035,0,3136,2494,1727477460,english,This is a great game. Everything is perfect ex...,False,17,1727202169,1727202169,"[-0.030018333345651627, 0.03668512776494026, -...","[-0.03265281021595001, -0.031750183552503586, ...","[-0.06248420476913452, -0.029605818912386894, ..."


In [25]:
import json
import os

# Define the data directory path
data_dir = os.path.abspath('../reviews-assistant/data/ground_truth')

# Define the input file path
input_file = os.path.join(data_dir, "ground_truth_retrieval.json")

# Check if the file exists before attempting to read
if os.path.exists(input_file):
    # Read the results from the JSON file
    with open(input_file, 'r', encoding='utf-8') as json_file:
        ground_truth = json.load(json_file)  # Load the JSON data into a Python object

    # Now ground_truth contains the data from ground_truth_retrieval.json
    print(f"Data loaded from {input_file}")
    print(f"Number of entries: {len(ground_truth)}")
    # Optionally display the first few records
    print(ground_truth[:5])  # Display the first 5 entries
else:
    print(f"File {input_file} does not exist!")

Data loaded from /home/jovyan/reviews-assistant/data/ground_truth/ground_truth_retrieval.json
Number of entries: 105
[{'appid': '2322010', 'question': 'How well does God of War: Ragnarok perform on PC at launch?'}, {'appid': '2322010', 'question': 'What kind of PC specifications did the reviewer use while playing God of War: Ragnarok?'}, {'appid': '2322010', 'question': 'What resolution and settings were used to play God of War: Ragnarok according to the review?'}, {'appid': '2322010', 'question': 'Did the reviewer experience any performance issues such as stutter while playing God of War: Ragnarok?'}, {'appid': '2322010', 'question': "Is there any mention of the game's story or gameplay quality in the review?"}]


In [26]:
ground_truth[:2]

[{'appid': '2322010',
  'question': 'How well does God of War: Ragnarok perform on PC at launch?'},
 {'appid': '2322010',
  'question': 'What kind of PC specifications did the reviewer use while playing God of War: Ragnarok?'}]

In [27]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [28]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [29]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [30]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score += 1 / (rank + 1)
                break  # Stop after finding the first relevant document

    return total_score / len(relevance_total)


In [31]:
def elastic_search_hybrid(field, query, vector, title):

    index_name='steam-reviews'
    
    knn_query = {
        "field": field,  # The vector field to search (e.g., "title_review_vector")
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "boost": 0.5,
        "filter": {
            "term": {
                "title": title  # Filter by "title" (game title)
            }
        }
    }

    keyword_query = {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["review^3", "title"],  # Replace with relevant fields
                    "type": "best_fields",
                    "boost": 0.5,
                }
            },
            "filter": {
                "term": {
                    "title": title  # Filter by "title" (game title)
                }
            }
        }
    }

    search_query = {
        "knn": knn_query,
        "query": keyword_query,
        "size": 5,
        "_source": ["review", "title", "appid"]  # Fields to return
    }

    es_results = es.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [32]:
def elastic_search_hybrid(field, query, vector, title):
    index_name = 'steam-reviews'

    # KNN Query
    knn_query = {
        "field": field,  # The vector field to search (e.g., "title_review_vector")
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "boost": 0.5
    }

    # Keyword Query
    keyword_query = {
        "bool": {
            "must": [
                {
                    "multi_match": {
                        "query": query,
                        "fields": ["review^3", "title"],  # Fields for keyword search
                        "type": "best_fields",
                        "boost": 0.5
                    }
                }
            ],
            "filter": {
                "term": {
                    "title": title  # Filter by game title
                }
            }
        }
    }

    # Execute the KNN search first
    knn_search_query = {
        "knn": knn_query,
        "_source": ["review", "title", "appid"],  # Fields to return
        "size": 5
    }

    try:
        knn_results = es.search(
            index=index_name,
            body=knn_search_query
        )
        knn_docs = [hit['_source'] for hit in knn_results['hits']['hits']]
    except Exception as e:
        print(f"An error occurred during KNN search: {e}")
        knn_docs = []

    # Execute the keyword search
    keyword_search_query = {
        "query": keyword_query,
        "_source": ["review", "title", "appid"],  # Fields to return
        "size": 5
    }

    try:
        keyword_results = es.search(
            index=index_name,
            body=keyword_search_query
        )
        keyword_docs = [hit['_source'] for hit in keyword_results['hits']['hits']]
    except Exception as e:
        print(f"An error occurred during keyword search: {e}")
        keyword_docs = []

    # Combine the results from both queries (KNN and keyword)
    combined_results = knn_docs + keyword_docs

    return combined_results


In [33]:
def elastic_search_hybrid(field, query, vector, appid):

    index_name='steam-reviews'
    
    knn_query = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "boost": 0.5,
        "filter": {
            "term": {
                "appid": appid
            }
        }
    }

    keyword_query = {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question", "appid"],
                    "type": "best_fields",
                    "boost": 0.5,
                }
            },
            "filter": {
                "term": {
                    "appid": appid
                }
            }
        }
    }

    # reviews:
    search_query = {
        "knn": knn_query,
        "query": keyword_query,
        "size": 5,
        "_source": ["appid",
        "timestamp_query",
        "title",
        "recommendationid",
        "author.steamid",
        "author.playtimeforever",
        "author.playtime_last_two_weeks",
        "author.playtime_at_review",
        "author.last_played",
        "language",
        "review",
        "voted_up",
        "votes_up",
        "timestamp_created",
        "timestamp_updated"
        ]
    }

    es_results = es.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [34]:
def question_hybrid(q):
    question = q['question']
    appid = q['appid']

    v_q = model.encode(question)

    return elastic_search_hybrid('review_vector', question, v_q, appid)

In [35]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['question']
        results = search_function(q)
        relevance = [d['appid'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [36]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        print(q)
        
        # doc_id = q['question']
        doc_id = q['appid']
        
        results = search_function(q)
        print(results[-1])
        
        relevance = [d['appid'] == doc_id for d in results]
        print(relevance[-1])
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [37]:
evaluate(ground_truth, question_hybrid)

  0%|          | 0/105 [00:00<?, ?it/s]

{'appid': '2322010', 'question': 'How well does God of War: Ragnarok perform on PC at launch?'}


  4%|▍         | 4/105 [00:00<00:03, 32.33it/s]

{'appid': '2322010', 'timestamp_query': 1727643597, 'title': 'God of War: Ragnarok', 'recommendationid': '175333633', 'author.steamid': '76561198056479550', 'author.playtimeforever': 0, 'author.playtime_last_two_weeks': 3966, 'author.playtime_at_review': 778, 'author.last_played': 1727458460, 'language': 'english', 'review': 'Ragnarok is more of everything that was great in the 2018 release, combined with the epic scale of the original trilogy. \nThe continuing story is outstanding, with thoroughly developed characters along with so much lore to chew on. Looks incredible and features once again brilliant, heartfelt voice acting (and mocap). Exceptionally well crafted environments and realms make exploration so fun, especially with the way they evolve throughout the story. \nCombat is as satisfying, brutal and rewarding as ever with some truly breathtaking boss fights. \nAn undeniably superlative experience.', 'voted_up': True, 'votes_up': 5, 'timestamp_created': 1726782601, 'timestamp_

  8%|▊         | 8/105 [00:00<00:02, 34.53it/s]

...

 97%|█████████▋| 102/105 [00:02<00:00, 44.27it/s]

{'appid': '1817070', 'timestamp_query': 1727643597, 'title': 'Marvel’s Spider-Man Remastered', 'recommendationid': '174988024', 'author.steamid': '76561199090152635', 'author.playtimeforever': 0, 'author.playtime_last_two_weeks': 1062, 'author.playtime_at_review': 430, 'author.last_played': 1727493791, 'language': 'english', 'review': "Love the game, but once you play the new Marvel's Spider-Man game, this one feels a little boring, no hate tho, still an amazing game!", 'voted_up': True, 'votes_up': 0, 'timestamp_created': 1726371208, 'timestamp_updated': 1726371208}
True
{'appid': '1817070', 'question': "What are the main highlights of the gameplay experience in Marvel's Spider-Man Remastered?"}
{'appid': '1817070', 'timestamp_query': 1727643597, 'title': 'Marvel’s Spider-Man Remastered', 'recommendationid': '175943463', 'author.steamid': '76561198013394552', 'author.playtimeforever': 0, 'author.playtime_last_two_weeks': 1598, 'author.playtime_at_review': 1598, 'author.last_played': 1

100%|██████████| 105/105 [00:02<00:00, 41.08it/s]

{'appid': '794540', 'timestamp_query': 1727643597, 'title': 'Neo Cab', 'recommendationid': '175331713', 'author.steamid': '76561198236511942', 'author.playtimeforever': 0, 'author.playtime_last_two_weeks': 215, 'author.playtime_at_review': 215, 'author.last_played': 1726779936, 'language': 'english', 'review': 'Very intricate visual novel with opportunities to "choose your fate". Rich characters, relevant, thoughtful narrative. RISE!!!!!', 'voted_up': True, 'votes_up': 0, 'timestamp_created': 1726780035, 'timestamp_updated': 1726780035}
True
{'appid': '794540', 'question': 'Is there a built-in system for saving progress in Neo Cab, and how does it work?'}
{'appid': '794540', 'timestamp_query': 1727643597, 'title': 'Neo Cab', 'recommendationid': '175041231', 'author.steamid': '76561199350825327', 'author.playtimeforever': 0, 'author.playtime_last_two_weeks': 123, 'author.playtime_at_review': 327, 'author.last_played': 1726422406, 'language': 'english', 'review': 'what a waster of time a

{'hit_rate': 1.0, 'mrr': 1.0}

In [38]:
def text_hybrid(q):
    question = q['question']
    appid = q['appid']

    v_q = model.encode(question)

    return elastic_search_hybrid('title_vector', question, v_q, appid)

In [39]:
evaluate(ground_truth, text_hybrid)

  0%|          | 0/105 [00:00<?, ?it/s]

{'appid': '2322010', 'question': 'How well does God of War: Ragnarok perform on PC at launch?'}


  5%|▍         | 5/105 [00:00<00:02, 42.59it/s]

{'appid': '2322010', 'timestamp_query': 1727643597, 'title': 'God of War: Ragnarok', 'recommendationid': '175347860', 'author.steamid': '76561198354466742', 'author.playtimeforever': 0, 'author.playtime_last_two_weeks': 827, 'author.playtime_at_review': 250, 'author.last_played': 1727081833, 'language': 'english', 'review': "to God of War: Ragnarok Dev. Team,\n\nI’m writing to bring to your attention an issue regarding the current PC port of God of War: Ragnarok that affects users with 4GB VRAM graphics cards. When attempting to launch the game on such systems, an error message is displayed, indicating that there is not enough VRAM, and the game closes without allowing users to proceed. It is my humble request to kindly reconsider or remove this VRAM limitation. From personal experience and community feedback, I can confirm that the game runs well on certain integrated graphics processors (such as the Ryzen 3 3200G, Ryzen 5 3400G, and Ryzen 7 5700G), which are generally weaker than man

 15%|█▌        | 16/105 [00:00<00:01, 44.84it/s]

...

 96%|█████████▌| 101/105 [00:02<00:00, 46.59it/s]

{'appid': '1496790', 'timestamp_query': 1727643597, 'title': 'Gotham Knights', 'recommendationid': '174266225', 'author.steamid': '76561198072033688', 'author.playtimeforever': 0, 'author.playtime_last_two_weeks': 0, 'author.playtime_at_review': 1180, 'author.last_played': 1725525394, 'language': 'english', 'review': '[h1]Superheroes in training[/h1]\n\nAn experience that might be enjoyable with a coop partner, but not worth it if you are going solo.\n\nRepetitiveness is the keyword. There are many activities or "patrols" as they call them, to gain levels and grind materials for equipment crafting. They all turn out pretty much samey, in that it all ends in an all out brawl, pummeling enemies that feel like punchbags, soaking a lot of damage. Yet these patrols are short and way too simplistic in design. More handcrafted experiences would\'ve been nice. It felt like a copy-paste each night.\n\nWithout knowing the actual intent of the designers, it felt like one of those MMORPGs that has

100%|██████████| 105/105 [00:02<00:00, 44.22it/s]

{'appid': '794540', 'timestamp_query': 1727643597, 'title': 'Neo Cab', 'recommendationid': '175310048', 'author.steamid': '76561199200313777', 'author.playtimeforever': 0, 'author.playtime_last_two_weeks': 204, 'author.playtime_at_review': 267, 'author.last_played': 1726757626, 'language': 'english', 'review': "I wasn't expecting much when I bought it, I thought I'd like the games vibes and the world, but the story was surpisingly very gripping :}\r\n\r\nindie games sure keep surprising me, and you can tell there was much love poured into this. def reccomend.", 'voted_up': True, 'votes_up': 0, 'timestamp_created': 1726757920, 'timestamp_updated': 1726757920}
True


{'hit_rate': 1.0, 'mrr': 1.0}

In [40]:
def question_text_hybrid(q):
    question = q['question']
    appid = q['appid']

    v_q = model.encode(question)

    return elastic_search_hybrid('title_review_vector', question, v_q, appid)

In [41]:
evaluate(ground_truth, question_text_hybrid)

  5%|▍         | 5/105 [00:00<00:02, 47.35it/s]

{'appid': '2322010', 'question': 'How well does God of War: Ragnarok perform on PC at launch?'}
{'appid': '2322010', 'timestamp_query': 1727643597, 'title': 'God of War: Ragnarok', 'recommendationid': '175333633', 'author.steamid': '76561198056479550', 'author.playtimeforever': 0, 'author.playtime_last_two_weeks': 3966, 'author.playtime_at_review': 778, 'author.last_played': 1727458460, 'language': 'english', 'review': 'Ragnarok is more of everything that was great in the 2018 release, combined with the epic scale of the original trilogy. \nThe continuing story is outstanding, with thoroughly developed characters along with so much lore to chew on. Looks incredible and features once again brilliant, heartfelt voice acting (and mocap). Exceptionally well crafted environments and realms make exploration so fun, especially with the way they evolve throughout the story. \nCombat is as satisfying, brutal and rewarding as ever with some truly breathtaking boss fights. \nAn undeniably superla

 14%|█▍        | 15/105 [00:00<00:01, 45.94it/s]

{'appid': '2369390', 'timestamp_query': 1727643597, 'title': 'Far Cry 6', 'recommendationid': '174446096', 'author.steamid': '76561199060873449', 'author.playtimeforever': 0, 'author.playtime_last_two_weeks': 0, 'author.playtime_at_review': 515, 'author.last_played': 1725852126, 'language': 'english', 'review': 'I never had an interest playing any Far Cry, Until I found the 6th Version and oh wow, I Play it everyday, The Graphics are insanely good, Story line is ADDICTING, The First/Third person shooter expierence is a must see!', 'voted_up': True, 'votes_up': 0, 'timestamp_created': 1725720357, 'timestamp_updated': 1725720357}
True
{'appid': '243470', 'question': 'Is the shooting mechanic in Watch_Dogs considered reliable and enjoyable by players?'}
{'appid': '243470', 'timestamp_query': 1727643597, 'title': 'Watch_Dogs', 'recommendationid': '175449438', 'author.steamid': '76561198883573243', 'author.playtimeforever': 0, 'author.playtime_last_two_weeks': 491, 'author.playtime_at_revie

 19%|█▉        | 20/105 [00:00<00:02, 38.86it/s]

...

 94%|█████████▍| 99/105 [00:02<00:00, 43.56it/s]

{'appid': '1496790', 'timestamp_query': 1727643597, 'title': 'Gotham Knights', 'recommendationid': '174955027', 'author.steamid': '76561198870001882', 'author.playtimeforever': 0, 'author.playtime_last_two_weeks': 0, 'author.playtime_at_review': 42, 'author.last_played': 1708628321, 'language': 'english', 'review': 'Very good quality game to play just need a better quality cpu for the game myself other than that good game and would recommend to anyone', 'voted_up': True, 'votes_up': 0, 'timestamp_created': 1726334425, 'timestamp_updated': 1726334425}
True
{'appid': '1817070', 'question': "Is Marvel's Spider-Man Remastered a well-optimized game upon release?"}
{'appid': '1817070', 'timestamp_query': 1727643597, 'title': 'Marvel’s Spider-Man Remastered', 'recommendationid': '175870064', 'author.steamid': '76561199101256665', 'author.playtimeforever': 0, 'author.playtime_last_two_weeks': 0, 'author.playtime_at_review': 2827, 'author.last_played': 1725051061, 'language': 'english', 'review

100%|██████████| 105/105 [00:02<00:00, 42.50it/s]

{'appid': '794540', 'timestamp_query': 1727643597, 'title': 'Neo Cab', 'recommendationid': '174022249', 'author.steamid': '76561197989390503', 'author.playtimeforever': 0, 'author.playtime_last_two_weeks': 0, 'author.playtime_at_review': 274, 'author.last_played': 1725145471, 'language': 'english', 'review': 'Giving it a thumbs up but with heavy reservations.\r\n\r\nNeo Cab has us take a role of Lina, a cab driver in a neo-futuristic city. She\'s just moved in to the city to live with her best friend from the past whom she hasn\'t seen in ages. However, on the first night, her friend goes missing and Lina takes it upon herself to find out what happened to her friend and who is responsible while at the same time trying to make a living by driving passengers.\r\n\r\nSo in a sense, Neo Cab is a hybrid of a visual novel story and management gameplay which involves serving customers, making enough money to get by, and making sure your star rating stays high. Normally, this kind of genre mix

{'hit_rate': 1.0, 'mrr': 1.0}

In [42]:
def compute_rrf(rank, k=60):
    """ Our own implementation of the relevance score """
    return 1 / (k + rank)

def elastic_search_hybrid_rrf(field, query, vector, appid, k=60):

    index_name = 'steam-reviews'
    
    knn_query = {
        "field": field,
        "query_vector": vector,
        "k": 10,
        "num_candidates": 10000,
        "boost": 0.5,
        "filter": {
            "term": {
                "appid": appid
            }
        }
    }

    keyword_query = {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question", "appid"],
                    "type": "best_fields",
                    "boost": 0.5,
                }
            },
            "filter": {
                "term": {
                    "appid": appid
                }
            }
        }
    }

    knn_results = es.search(
        index=index_name, 
        body={
            "knn": knn_query, 
            "size": 10
        }
    )['hits']['hits']
    
    keyword_results = es.search(
        index=index_name, 
        body={
            "query": keyword_query, 
            "size": 10
        }
    )['hits']['hits']
    
    rrf_scores = {}
    # Calculate RRF using vector search results
    for rank, hit in enumerate(knn_results):
        doc_id = hit['_id']
        rrf_scores[doc_id] = compute_rrf(rank + 1, k)

    # Adding keyword search result scores
    for rank, hit in enumerate(keyword_results):
        doc_id = hit['_id']
        if doc_id in rrf_scores:
            rrf_scores[doc_id] += compute_rrf(rank + 1, k)
        else:
            rrf_scores[doc_id] = compute_rrf(rank + 1, k)

    # Sort RRF scores in descending order
    reranked_docs = sorted(rrf_scores.items(), key=lambda x: x[1], reverse=True)
    
    # Get top-K documents by the score
    final_results = []
    for doc_id, score in reranked_docs[:5]:
        doc = es.get(index=index_name, id=doc_id)
        final_results.append(doc['_source'])
    
    return final_results

In [43]:
def question_text_hybrid_rrf(q):
    question = q['question']
    appid = q['appid']

    v_q = model.encode(question)

    return elastic_search_hybrid_rrf('title_review_vector', question, v_q, appid)

evaluate(ground_truth, question_text_hybrid_rrf)

  0%|          | 0/105 [00:00<?, ?it/s]

{'appid': '2322010', 'question': 'How well does God of War: Ragnarok perform on PC at launch?'}


  3%|▎         | 3/105 [00:00<00:04, 21.05it/s]

{'appid': '2322010', 'timestamp_query': 1727643597, 'title': 'God of War: Ragnarok', 'recommendationid': '175333633', 'author.steamid': '76561198056479550', 'author.playtimeforever': 0, 'author.playtime_last_two_weeks': 3966, 'author.playtime_at_review': 778, 'author.last_played': 1727458460, 'language': 'english', 'review': 'Ragnarok is more of everything that was great in the 2018 release, combined with the epic scale of the original trilogy. \nThe continuing story is outstanding, with thoroughly developed characters along with so much lore to chew on. Looks incredible and features once again brilliant, heartfelt voice acting (and mocap). Exceptionally well crafted environments and realms make exploration so fun, especially with the way they evolve throughout the story. \nCombat is as satisfying, brutal and rewarding as ever with some truly breathtaking boss fights. \nAn undeniably superlative experience.', 'voted_up': True, 'votes_up': 5, 'timestamp_created': 1726782601, 'timestamp_

  6%|▌         | 6/105 [00:00<00:04, 21.13it/s]

...

 97%|█████████▋| 102/105 [00:04<00:00, 24.90it/s]

{'appid': '794540', 'timestamp_query': 1727643597, 'title': 'Neo Cab', 'recommendationid': '175041231', 'author.steamid': '76561199350825327', 'author.playtimeforever': 0, 'author.playtime_last_two_weeks': 123, 'author.playtime_at_review': 327, 'author.last_played': 1726422406, 'language': 'english', 'review': 'what a waster of time and she made me drive 6 days', 'voted_up': False, 'votes_up': 3, 'timestamp_created': 1726422456, 'timestamp_updated': 1726422456, 'title_vector': [0.003824847750365734, -0.0463419035077095, -0.02124926820397377, -0.006213557906448841, -0.02198488637804985, -0.00460848631337285, -0.036455366760492325, 0.031095093116164207, -0.05228733643889427, -0.008822431787848473, 0.018506495282053947, 0.08161478489637375, 0.03398526459932327, 0.01097057294100523, -0.022859830409288406, 0.043719250708818436, 0.041203975677490234, -0.05641128495335579, 0.1188468188047409, 0.030031869187951088, 0.02119450457394123, -0.07324086129665375, -0.018102973699569702, 0.04289178922

100%|██████████| 105/105 [00:04<00:00, 24.81it/s]

{'appid': '794540', 'timestamp_query': 1727643597, 'title': 'Neo Cab', 'recommendationid': '175171850', 'author.steamid': '76561198024806714', 'author.playtimeforever': 0, 'author.playtime_last_two_weeks': 227, 'author.playtime_at_review': 235, 'author.last_played': 1726577013, 'language': 'english', 'review': "Cyberpunk Uber simulator but as a San Francisco visual novel set in cyber Los Ochos.  It's obviously written by Leigh Alexander and her narrative voice shines through.  The graphics are beautiful especially for something made in Unity.  The atmosphere and music is nice although it can be slow at times.  You're basically exploring the narratives of your customers which are delivered episodically.  The final emotional battle between Savvy and Lina owes a lot to the insult sword fights from Monkey Island.\n\nWorth playing if you've got it in your backlog and worth a buy if it's 85% off.  It'll last you a quick night or two.  It just kind of ends without anything visible occurring o

{'hit_rate': 1.0, 'mrr': 1.0}

In [44]:
index = minsearch.Index(
    # text_fields=["author.steamid", "author.playtimeforever", "author.playtime_last_two_weeks", "author.playtime_at_review", "author.last_played", "language", "review", "timestamp_created", "timestamp_updated"],
    text_fields=["title", "language", "review"],
    keyword_fields=["appid", "recommendationid"]
)

index.fit(reviews)

NameError: name 'minsearch' is not defined

# Retrieval evaluation

In [ ]:
# Directory containing the ground truth file
data_dir = os.path.abspath('../reviews-assistant/data/ground_truth')

# Path to the ground_truth_retrieval.json file
file_path = os.path.join(data_dir, 'ground_truth_retrieval.json')

# Check if the file exists
if os.path.exists(file_path):
    try:
        # Open and load the JSON file
        with open(file_path, 'r', encoding='utf-8') as jsonfile:
            ground_truth_data = json.load(jsonfile)  # Load the JSON data into a Python object

        # Ensure each item is a dictionary and contains 'id' and 'question'
        if all(isinstance(item, dict) and 'appid' in item and 'question' in item for item in ground_truth_data):
            print("Data successfully loaded and is in the correct format.")

            # Example: Print a sample of the data
            for item in ground_truth_data[:5]:  # Print the first 5 questions
                print(f"ID: {item['appid']}, Question: {item['question']}")
        else:
            print("Error: The data format is incorrect or missing required fields ('id', 'question').")
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
else:
    print(f"File {file_path} does not exist.")

In [ ]:
ground_truth_data[:1]

In [ ]:
for doc in tqdm(reviews):
    question = doc['review']
    text = doc['text']
    qt = question + ' ' + text

    doc['question_vector'] = model.encode(question)
    doc['text_vector'] = model.encode(text)
    doc['question_text_vector'] = model.encode(qt)

In [ ]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [ ]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score += 1 / (rank + 1)
                break  # Stop after finding the first relevant document

    return total_score / len(relevance_total)


In [ ]:
def minsearch_search(query):
    boost = {}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

In [ ]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['appid']
        results = search_function(q)
        relevance = [d['appid'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [ ]:
from tqdm.auto import tqdm

In [ ]:
evaluate(ground_truth_data, lambda q: minsearch_search(q['question']))

# Finding the best parameters

In [ ]:
len(ground_truth_data)

In [ ]:
# Find the midpoint of the dataset
midpoint = len(ground_truth_data) // 2  # integer division to get the midpoint

# Split the dataset into two equal halves
df_validation = ground_truth_data[:midpoint]
df_test = ground_truth_data[midpoint:]

In [ ]:
df_validation[-1]

In [ ]:
df_test[-1]

In [ ]:
import random

def simple_optimize(param_ranges, objective_function, n_iterations=10):
    best_params = None
    best_score = float('-inf')  # Assuming we're minimizing. Use float('-inf') if maximizing.

    for _ in range(n_iterations):
        # Generate random parameters
        current_params = {}
        for param, (min_val, max_val) in param_ranges.items():
            if isinstance(min_val, int) and isinstance(max_val, int):
                current_params[param] = random.randint(min_val, max_val)
            else:
                current_params[param] = random.uniform(min_val, max_val)
        
        # Evaluate the objective function
        current_score = objective_function(current_params)
        
        # Update best if current is better
        if current_score > best_score:  # Change to > if maximizing
            best_score = current_score
            best_params = current_params
    
    return best_params, best_score

In [ ]:
def minsearch_search(query, boost=None):
    if boost is None:
        boost = {}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

In [ ]:
param_ranges = {
    'exercise_name': (0.0, 3.0),
    'type_of_activity': (0.0, 3.0),
    'type_of_equipment': (0.0, 3.0),
    'body_part': (0.0, 3.0),
    'type': (0.0, 3.0),
    'muscle_groups_activated': (0.0, 3.0),
    'instructions': (0.0, 3.0),
}

def objective(boost_params):
    def search_function(q):
        return minsearch_search(q['question'], boost_params)

    results = evaluate(df_validation, search_function)
    return results['mrr']

In [ ]:
simple_optimize(param_ranges, objective, n_iterations=20)

In [ ]:
def minsearch_improved(query):
    boost = {'exercise_name': 1.5333100039172263,
  'type_of_activity': 1.5395670128097776,
  'type_of_equipment': 2.39211934690812,
  'body_part': 1.6244459844173096,
  'type': 2.7948598963438456,
  'muscle_groups_activated': 1.4303057412632778,
  'instructions': 1.46575166599529}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

evaluate(ground_truth_data, lambda q: minsearch_improved(q['question']))